# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [15]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15992904888560757500
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357564416
locality {
  bus_id: 1
}
incarnation: 16137360452278271474
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [7]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [12]:
?Tokenizer.fit_on_texts


In [6]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)

    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))
    

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [22]:
#english_sentences_tk, english_tokenizer = tokenize(english_sentences)
#french_sentences_tk, french_tokenizer = tokenize(french_sentences)
print("# english sentences", len(english_sentences_tk))
eng_maxlen = max([len(s) for s in english_sentences_tk])
print("english maxlen of tokens seqs", eng_maxlen)
print("# french sentences", len(french_sentences_tk))
fr_maxlen = max([len(s) for s in french_sentences_tk])
print("french maxlen of tokens seqs", fr_maxlen)

### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [9]:
?pad_sequences # padding='pre'

In [7]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if not length: length = max([len(s) for s in x])
    return pad_sequences(x, maxlen=length ,padding='post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [8]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
max(french_tokenizer.word_index.values())

344

In [9]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [20]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    GRU_output_size = 2**9 #max(french_vocab_size, english_vocab_size) + 1
    
    inputs = Input(shape=input_shape[1:])
    x = GRU(GRU_output_size, return_sequences=True, unroll=True)(inputs)
    ## testing some additional dense layers, adds few % of accuracy
    # x = Dropout(0.25)(x)
    # dense_1 = Dense(GSR_output_size, activation='relu')
    # x = TimeDistributed(dense_1)(x)
    # dense_2 = Dense(64, activation='relu')
    # x = TimeDistributed(dense_2)(x)
    logit = Dense(french_vocab_size, activation='softmax')
    outputs = TimeDistributed(logit)(x)
    
    model = Model(inputs, outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)

In [17]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x[:,:,np.newaxis]
print(tmp_x.shape)

(137861, 21, 1)


In [18]:
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)
print(simple_rnn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_3 (GRU)                  (None, 21, 512)           789504    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 345)           176985    
Total params: 966,489
Trainable params: 966,489
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
from workspace_utils import active_session
with active_session():
    simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.15)

# Print prediction(s)
word_lookup = {idx: w for w, idx in english_tokenizer.word_index.items()}
word_lookup.update({0: '<PAD>'})
print(' '.join([word_lookup[i[0]] for i in tmp_x[:1][0]]))
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

In [33]:
# # with GRU output size = 512 + logit

Train on 117181 samples, validate on 20680 samples
Epoch 1/10
117181/117181 [==============================] - 26s 221us/step - loss: 2.2046 - acc: 0.5239 - val_loss: 1.5620 - val_acc: 0.5925
Epoch 2/10
117181/117181 [==============================] - 25s 210us/step - loss: 1.4180 - acc: 0.6153 - val_loss: 1.3090 - val_acc: 0.6308
Epoch 3/10
117181/117181 [==============================] - 25s 210us/step - loss: 1.2399 - acc: 0.6404 - val_loss: 1.1734 - val_acc: 0.6528
Epoch 4/10
117181/117181 [==============================] - 25s 210us/step - loss: 1.1289 - acc: 0.6603 - val_loss: 1.0822 - val_acc: 0.6696
Epoch 5/10
117181/117181 [==============================] - 25s 210us/step - loss: 1.0536 - acc: 0.6727 - val_loss: 1.0225 - val_acc: 0.6788
Epoch 6/10
117181/117181 [==============================] - 25s 210us/step - loss: 0.9993 - acc: 0.6807 - val_loss: 0.9793 - val_acc: 0.6823
Epoch 7/10
117181/117181 [==============================] - 25s 210us/step - loss: 0.9555 - acc: 0.6884

In [30]:
# with GRU output size = 512 + 0.25 drop out + dense layer to 512 + logit

Train on 117181 samples, validate on 20680 samples
Epoch 1/10
117181/117181 [==============================] - 30s 259us/step - loss: 2.1564 - acc: 0.5211 - val_loss: 1.4612 - val_acc: 0.6087
Epoch 2/10
117181/117181 [==============================] - 29s 247us/step - loss: 1.3552 - acc: 0.6131 - val_loss: 1.2215 - val_acc: 0.6353
Epoch 3/10
117181/117181 [==============================] - 29s 248us/step - loss: 1.1774 - acc: 0.6448 - val_loss: 1.0744 - val_acc: 0.6761
Epoch 4/10
117181/117181 [==============================] - 29s 248us/step - loss: 1.0678 - acc: 0.6667 - val_loss: 0.9822 - val_acc: 0.6920
Epoch 5/10
117181/117181 [==============================] - 29s 248us/step - loss: 0.9995 - acc: 0.6792 - val_loss: 0.9229 - val_acc: 0.7022
Epoch 6/10
117181/117181 [==============================] - 29s 248us/step - loss: 0.9460 - acc: 0.6899 - val_loss: 0.8815 - val_acc: 0.7124
Epoch 7/10
117181/117181 [==============================] - 29s 248us/step - loss: 0.9020 - acc: 0.6989

In [14]:
# with 345 GSR output size + logit output

Train on 117181 samples, validate on 20680 samples
Epoch 1/10
117181/117181 [==============================] - 20s 167us/step - loss: 2.3774 - acc: 0.5075 - val_loss: 1.6726 - val_acc: 0.5809
Epoch 2/10
117181/117181 [==============================] - 18s 150us/step - loss: 1.5076 - acc: 0.6023 - val_loss: 1.3958 - val_acc: 0.6152
Epoch 3/10
117181/117181 [==============================] - 18s 149us/step - loss: 1.3221 - acc: 0.6294 - val_loss: 1.2533 - val_acc: 0.6398
Epoch 4/10
117181/117181 [==============================] - 18s 150us/step - loss: 1.2060 - acc: 0.6498 - val_loss: 1.1630 - val_acc: 0.6568
Epoch 5/10
117181/117181 [==============================] - 18s 150us/step - loss: 1.1265 - acc: 0.6630 - val_loss: 1.0930 - val_acc: 0.6734
Epoch 6/10
117181/117181 [==============================] - 18s 150us/step - loss: 1.0672 - acc: 0.6721 - val_loss: 1.0473 - val_acc: 0.6789
Epoch 7/10
117181/117181 [==============================] - 18s 150us/step - loss: 1.0227 - acc: 0.6770

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [12]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # TODO: Implement
    learning_rate = 1e-3
    embedding_size = 512 # max(french_vocab_size, english_vocab_size) 
    GRU_size = 512 # max(french_vocab_size, english_vocab_size) 
    
    inputs = Input(shape=input_shape[1:])
    embedded = Embedding(english_vocab_size, embedding_size, input_length=input_shape[1])(inputs)
    x = GRU(GRU_size, return_sequences=True, unroll=True)(embedded)
    logit = Dense(french_vocab_size, activation='softmax')
    outputs = TimeDistributed(logit)(x)
    
    model = Model(inputs, outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

tests.test_embed_model(embed_model)

In [14]:
# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# when imbedding is used, no new axis needs added to input, since embedding adds new axis
#tmp_x = tmp_x[:,:,np.newaxis]
print(tmp_x.shape)

(137861, 21)


In [15]:
# TODO: Train the neural network

# prepare model for training
embedding_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)
print(embedding_rnn_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 21, 512)           102400    
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 512)           1574400   
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 345)           176985    
Total params: 1,853,785
Trainable params: 1,853,785
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
# Train model
from workspace_utils import active_session
with active_session():
    embedding_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.15)

# TODO: Print prediction(s)
word_lookup = {idx: w for w, idx in english_tokenizer.word_index.items()}
word_lookup.update({0: '<PAD>'})
print(' '.join([word_lookup[i] for i in tmp_x[:1][0]]))
print(logits_to_text(embedding_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 117181 samples, validate on 20680 samples
Epoch 1/10
117181/117181 [==============================] - 28s 238us/step - loss: 2.4606 - acc: 0.5304 - val_loss: 1.3074 - val_acc: 0.6927
Epoch 2/10
117181/117181 [==============================] - 26s 218us/step - loss: 0.8629 - acc: 0.7796 - val_loss: 0.5936 - val_acc: 0.8263
Epoch 3/10
117181/117181 [==============================] - 26s 220us/step - loss: 0.4807 - acc: 0.8540 - val_loss: 0.3884 - val_acc: 0.8777
Epoch 4/10
117181/117181 [==============================] - 26s 221us/step - loss: 0.3469 - acc: 0.8899 - val_loss: 0.3068 - val_acc: 0.9015
Epoch 5/10
117181/117181 [==============================] - 26s 221us/step - loss: 0.2876 - acc: 0.9064 - val_loss: 0.2705 - val_acc: 0.9111
Epoch 6/10
117181/117181 [==============================] - 26s 221us/step - loss: 0.2514 - acc: 0.9170 - val_loss: 0.2399 - val_acc: 0.9203
Epoch 7/10
117181/117181 [==============================] - 26s 221us/step - loss: 0.2283 - acc: 0.9237

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [41]:
# TESTING BD MODEL WITH AND WITHOUT EMBEDDING LAYER 
# EMBEDDING TURNS OUT TO IMPROVE ACCURACY from tremendously to 98% from around 80% without embedding

model_uses_embedding = False

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 2.5e-3
    embedding_size = 512 
    GRU_size = 512 
    
    inputs = Input(shape=input_shape[1:])
    if model_uses_embedding:
        x = Embedding(english_vocab_size, embedding_size, input_length=input_shape[1])(inputs)
    else:
        x = inputs 
    GRU_layer = GRU(GRU_size, return_sequences=True, unroll=True)
    x = Bidirectional(GRU_layer)(x)
    x = Dropout(0.3)(x)
    logit = Dense(french_vocab_size, activation='softmax')
    outputs = TimeDistributed(logit)(x)
    
    model = Model(inputs, outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
 
    return model
# test does not pass with embedding on bd model
if not model_uses_embedding:
    tests.test_bd_model(bd_model)
# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# when imbedding is used, no new axis needs added to input, since embedding adds new axis
if not model_uses_embedding:
    tmp_x = tmp_x[:,:,np.newaxis]
print(tmp_x.shape)
# prepare model for training
bd_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)
print(bd_model.summary())


(137861, 21, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 21, 1024)          1579008   
_________________________________________________________________
dropout_7 (Dropout)          (None, 21, 1024)          0         
_________________________________________________________________
time_distributed_9 (TimeDist (None, 21, 345)           353625    
Total params: 1,932,633
Trainable params: 1,932,633
Non-trainable params: 0
_________________________________________________________________
None


In [42]:
# TODO: Train and Print prediction(s)
from workspace_utils import active_session
with active_session():
    bd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)
# TODO: Print prediction(s)
word_lookup = {idx: w for w, idx in english_tokenizer.word_index.items()}
word_lookup.update({0: '<PAD>'})
if model_uses_embedding:
    print("model with embedding")
    print(' '.join([word_lookup[i] for i in tmp_x[:1][0]]))
else:
    print("no embedding")
    print(' '.join([word_lookup[i[0]] for i in tmp_x[:1][0]]))
print(logits_to_text(bd_model.predict(tmp_x[:1])[0], french_tokenizer))
      

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 37s 333us/step - loss: 1.6738 - acc: 0.5846 - val_loss: 1.1982 - val_acc: 0.6494
Epoch 2/10
110288/110288 [==============================] - 33s 303us/step - loss: 1.1450 - acc: 0.6558 - val_loss: 1.0142 - val_acc: 0.6829
Epoch 3/10
110288/110288 [==============================] - 33s 304us/step - loss: 1.0138 - acc: 0.6806 - val_loss: 0.9251 - val_acc: 0.7001
Epoch 4/10
110288/110288 [==============================] - 33s 303us/step - loss: 0.9430 - acc: 0.6922 - val_loss: 0.8628 - val_acc: 0.7102
Epoch 5/10
110288/110288 [==============================] - 33s 304us/step - loss: 0.8793 - acc: 0.7044 - val_loss: 0.7991 - val_acc: 0.7199
Epoch 6/10
110288/110288 [==============================] - 33s 304us/step - loss: 0.8399 - acc: 0.7120 - val_loss: 0.7715 - val_acc: 0.7332
Epoch 7/10
110288/110288 [==============================] - 34s 305us/step - loss: 0.7791 - acc: 0.7327

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 48s 436us/step - loss: 0.0583 - acc: 0.9815 - val_loss: 0.0656 - val_acc: 0.9807
Epoch 2/10
110288/110288 [==============================] - 48s 438us/step - loss: 0.0502 - acc: 0.9838 - val_loss: 0.0666 - val_acc: 0.9809
Epoch 3/10
110288/110288 [==============================] - 48s 439us/step - loss: 0.0453 - acc: 0.9852 - val_loss: 0.0641 - val_acc: 0.9821
Epoch 4/10
110288/110288 [==============================] - 48s 439us/step - loss: 0.0424 - acc: 0.9861 - val_loss: 0.0636 - val_acc: 0.9820
Epoch 5/10
110288/110288 [==============================] - 49s 442us/step - loss: 0.0402 - acc: 0.9869 - val_loss: 0.0641 - val_acc: 0.9828
Epoch 6/10
110288/110288 [==============================] - 48s 438us/step - loss: 0.0380 - acc: 0.9875 - val_loss: 0.0651 - val_acc: 0.9826
Epoch 7/10
110288/110288 [==============================] - 48s 439us/step - loss: 0.0465 - acc: 0.9854

### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

my Encoder - Decoder implementation is following explanation and adapting some code from https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

In [18]:

embedding_size = 2**9 
context_size = 2**9

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 2.5e-3
    # inputs of encoder (english sentences) and decoder (french sentences)
    inputs_enc = Input(shape=input_shape[1:])
    inputs_dec = Input(shape=input_shape[1:])
    # ENCODER 
    embedding_enc = Embedding(english_vocab_size, embedding_size)
    emb_enc = embedding_enc(inputs_enc)
    enc_gru = GRU(context_size, return_state=True, unroll=False)
    enc_out, context = enc_gru(emb_enc)
    # DECODER 
    embedding_dec = Embedding(french_vocab_size+1, embedding_size)
    emb_dec = embedding_dec(inputs_dec)
    decoder = GRU(context_size, return_sequences=True, return_state=True, unroll=False)
    dec_seq_out, dec_state = decoder(emb_dec, initial_state=context)
    logit = Dense(french_vocab_size, activation='softmax')
    outputs = TimeDistributed(logit)(dec_seq_out)
    # create full model and sub-components needed for inference
    full_model = Model([inputs_enc, inputs_dec], outputs)
    encoder = Model(inputs_enc, context)
    decoder_embedding = Model(inputs_dec, emb_dec)
    
    full_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    
    return full_model, encoder, decoder_embedding, decoder, logit

tests.test_encdec_model(encdec_model)

ValueError: Input 0 is incompatible with layer gru_3: expected ndim=3, found ndim=4

In [13]:
# PREPARE INPUT
# decoder input are also the labels sequences with additional <GO> as first token
# set <GO> to id = french_vocab_size+1 
offset_french_sentences = np.hstack((np.repeat(french_vocab_size+1,preproc_french_sentences.shape[0])[:, np.newaxis], 
                                     preproc_french_sentences[:,:,0]))
print(offset_french_sentences.shape)
# update other date with extended label sequences
max_french_sequence_length = offset_french_sentences.shape[1]
# input and lable sequences to be padded to fit extended seq length
target_french_sentences = pad(preproc_french_sentences, max_french_sequence_length)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)

# CREATE MODEL INSTANCE
(encdec_model,
 encoder,
 decoder_embedding, 
 decoder, 
 logit) = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size + 1,
    french_vocab_size + 1)
print(encdec_model.summary())

(137861, 22)
(137861, 22)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 22, 512)      102400      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 22, 512)      177152      input_2[0][0]                    
___________________________________________________________________________________

In [17]:
# TODO: Train and Print prediction(s)

training_input = [tmp_x, offset_french_sentences]

from workspace_utils import active_session
with active_session():
    encdec_model.fit(training_input, target_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 48s 432us/step - loss: 0.0270 - acc: 0.9924 - val_loss: 0.0262 - val_acc: 0.9930
Epoch 2/10
110288/110288 [==============================] - 48s 434us/step - loss: 0.0219 - acc: 0.9939 - val_loss: 0.0232 - val_acc: 0.9936
Epoch 3/10
110288/110288 [==============================] - 48s 435us/step - loss: 0.0186 - acc: 0.9949 - val_loss: 0.0207 - val_acc: 0.9945
Epoch 4/10
110288/110288 [==============================] - 48s 435us/step - loss: 0.0160 - acc: 0.9957 - val_loss: 0.0183 - val_acc: 0.9949
Epoch 5/10
110288/110288 [==============================] - 48s 434us/step - loss: 0.0136 - acc: 0.9964 - val_loss: 0.0166 - val_acc: 0.9954
Epoch 6/10
110288/110288 [==============================] - 48s 437us/step - loss: 0.0124 - acc: 0.9967 - val_loss: 0.0148 - val_acc: 0.9961
Epoch 7/10
110288/110288 [==============================] - 48s 434us/step - loss: 0.0108 - acc: 0.9971

#### INFERENCE  
for inference the model components have to be rearranged and predict word by word, where the prediction at step t is input to the decoder in step t + 1
(following explanation and adapting code from https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

In [ ]:
# REARRANGE the MODEL for INFERENCE

context = Input(shape=(context_size, ))
emb_input = Input(shape=(1,))

emb_dec = decoder_embedding(emb_input)
dec_out, decoder_state = decoder(emb_dec, initial_state=context)
decoder_output = logit(dec_out)
decoder_model = Model([emb_input, context], [decoder_output, decoder_state])

In [ ]:
# FUNCTION TO RUN REARRANGED MODEL STEP BY STEP TO PREDICT OUTPUT SEQUENCE
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder.predict(input_seq)

    # First 'predicted' token is set to <GO>
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = french_vocab_size+1

    # Sampling loop for a sequence
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        # predict tokes for output sequence one by one
        output_tokens, dec_state = decoder_model.predict(
            [target_seq, states_value])
        decoded_sentence.append(output_tokens)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
    
        # Exit condition
        if len(decoded_sentence) >= max_french_sequence_length:
            stop_condition = True

        # reshape predicted token for input to next prediction step
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = dec_state

    return decoded_sentence

In [22]:
# ADJUSTED LOGIT TO TEXT WITH additional <GO> token
def encdec_logits_to_text(logits_list, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    index_to_words[max(index_to_words)+1] = '<GO>'

    return ' '.join([index_to_words[np.argmax(prediction)] for prediction in logits_list])

print('`encdec_logits_to_text` function loaded.')

In [23]:
# USE THE INFERENCE MODEL TO TRANSLATE 

predicted = decode_sequence(tmp_x[:1])
word_lookup = {idx: w for w, idx in english_tokenizer.word_index.items()}
word_lookup.update({0: '<PAD>'})
print(' '.join([word_lookup[i] for i in tmp_x[:1][0]]))
print(encdec_logits_to_text(predicted, french_tokenizer))

new jersey is sometimes quiet during autumn and it is snowy in april <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 2.5e-3
    # inputs of encoder (english sentences) and decoder (french sentences)
    inputs_enc = Input(shape=input_shape[1:])
    inputs_dec = Input(shape=input_shape[1:])
    # ENCODER 
    embedding_enc = Embedding(english_vocab_size, embedding_size)
    emb_enc = embedding_enc(inputs_enc)
    enc_gru = GRU(context_size, return_state=True, unroll=False)
    enc_out, context = enc_gru(emb_enc)
    # DECODER 
    embedding_dec = Embedding(french_vocab_size+1, embedding_size)
    emb_dec = embedding_dec(inputs_dec)
    decoder = GRU(context_size, return_sequences=True, return_state=True, unroll=False)
    dec_seq_out, dec_state = decoder(emb_dec, initial_state=context)
    logit = Dense(french_vocab_size, activation='softmax')
    outputs = TimeDistributed(logit)(dec_seq_out)
    # create full model and sub-components needed for inference
    full_model = Model([inputs_enc, inputs_dec], outputs)
    encoder = Model(inputs_enc, context)
    decoder_embedding = Model(inputs_dec, emb_dec)
    
    full_model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    
    return full_model, encoder, decoder_embedding, decoder, logit

tests.test_model_final(model_final)
print('Final Model Loaded')

In [ ]:
# TODO: Train the final model

## Prediction (IMPLEMENTATION)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = None

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

## Submission
When you're ready to submit, complete the following steps:
1. Review the rubric to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [2]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 305996 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?